# Course of Action
1. Clean dataset
2. Scale dataset
3. Split dataset

In [115]:
# Importing the dataset
import pandas

dataset = pandas.read_csv("./AmesHousing.csv");

In [116]:
# Removing irrelevant values for the dataset
dataset = dataset.drop("PID", axis=1).drop("Order", axis=1)

In [117]:
# Enumerating categorical values
categories = {}
for column in dataset.columns:
	if dataset[column].dtype == 'object':
		dataset[column], categories[column] = dataset[column].factorize()
		# Normalize stuff so Null values don't become -1
		dataset[column] += 1

,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,20,1,141.0,31770,1,0,1,1,1,1,...,0,0,0,0,0,5,2010,1,1,215000
1,20,2,80.0,11622,1,0,2,1,1,2,...,0,0,1,0,0,6,2010,1,1,105000
2,20,1,81.0,14267,1,0,1,1,1,1,...,0,0,0,1,12500,6,2010,1,1,172000
3,20,1,93.0,11160,1,0,2,1,1,1,...,0,0,0,0,0,4,2010,1,1,244000
4,60,1,74.0,13830,1,0,1,1,1,2,...,0,0,1,0,0,3,2010,1,1,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,80,1,37.0,7937,1,0,1,1,1,3,...,0,0,2,0,0,3,2006,1,1,142500
2926,20,1,NaN,8885,1,0,1,4,1,2,...,0,0,1,0,0,6,2006,1,1,131000
2927,85,1,62.0,10441,1,0,2,1,1,2,...,0,0,1,2,700,7,2006,1,1,132000
2928,20,1,77.0,10010,1,0,2,1,1,2,...,0,0,0,0,0,4,2006,1,1,170000


In [118]:
# Checking the correlation between the different columns and the sales price
correlation = dataset.corr();
print(correlation["SalePrice"].sort_values(ascending=False).to_string())

SalePrice          1.000000
Overall Qual       0.799262
Gr Liv Area        0.706780
Garage Cars        0.647877
Garage Area        0.640401
Total Bsmt SF      0.632280
1st Flr SF         0.621676
Exter Qual         0.560112
Year Built         0.558426
Full Bath          0.545604
Year Remod/Add     0.532974
Garage Yr Blt      0.526965
Mas Vnr Area       0.508285
TotRms AbvGrd      0.495474
Kitchen Qual       0.489285
Bsmt Qual          0.482193
Fireplaces         0.474558
BsmtFin SF 1       0.432914
Lot Frontage       0.357318
Wood Deck SF       0.327143
Mas Vnr Type       0.318778
Fireplace Qu       0.316369
Open Porch SF      0.312951
Half Bath          0.285056
Bsmt Full Bath     0.276050
2nd Flr SF         0.269373
Lot Area           0.266549
Heating QC         0.199995
Bsmt Unf SF        0.182855
BsmtFin Type 1     0.166426
Bedroom AbvGr      0.143913
Screen Porch       0.112151
Pool Area          0.068403
Land Slope         0.066396
Roof Matl          0.063571
Garage Qual        0

In [120]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)
print(f"{len(train_set)} train instances + {len(test_set)} test instances")
"{hello} {world}".format(hello="Stuff", world=4)

2344 train instances + 586 test instances


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

In [119]:
from sklearn.linear_model import LinearRegression

# Using "Overall Qual" as an identifier for price due to its high correlation
lin_reg = LinearRegression()
lin_reg.fit()

TypeError: LinearRegression.fit() missing 2 required positional arguments: 'X' and 'y'